<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<b>CET Template Notebook</b> <br>
Contact author(s): <i>Author Name</i> <br>
Last verified to run: <i>yyyy-mm-dd</i> <br>
LSST Science Piplines version: Weekly <i>yyyy_xx</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>beginner</i> <br>

_In this template, text in italics are examples or instructions that should be: (a) removed if it is not applicable to the notebook; or (b) replaced with text that is appropriate for the notebook. But bold or regular text should appear pretty much as-is in all CET notebooks. For more information, see the [CET's Guidelines for Tutorial Notebooks](https://confluence.lsstcorp.org/pages/viewpage.action?pageId=168857070)._

_While developing, use the following code cell to check that the code conforms to standards, but then delete the cell and "Kernel --> Restart Kernel and Clear All Outputs" before saving and committing._

In [ ]:
# %load_ext pycodestyle_magic
# %flake8_on
# import logging
# logging.getLogger("flake8").setLevel(logging.FATAL)

_The six cells below are considered the extended header of the notebook. The first four will be used, verbatim, to create the table of notebook metadata in the README.md file for the repository._

**Description:** _Very brief description of notebook._

**Skills:** _Brief list of skills to match the README.md file for the repository._

**LSST Data Products:** _List the all of the types of LSST catalogs and images used._

**Packages:** _List the python packages used._ (_List the packages being taught first, e.g., afwDisplay for a notebook about displaying images. Then supporting packages, e.g., lsst.daf.butler for a notebook about displaying images. It is OK to leave out basic support packages like os or glob.)_

**Credit:**
_E.g., "Originally developed by" or "Based on notebooks developed by" and then people's names, including journal article or software release citations if appropriate._
Please consider acknowledging them if this notebook is used for the preparation of journal articles, software releases, or other notebooks.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0-2.lsst.io">dp0-2.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

_Provide a light narrative about this notebook, e.g., "This notebook will teach the user..."._

_Cite or link to any external information or documentation, and cross-reference to other notebooks._

### 1.1 Package Imports

_All package imports should be done in the first code cell._

_Provide explanation or external links to package documentation, where appropriate._

_E.g., Numpy is a fundamental package for scientific computing with arrays in Python (<a href="https://numpy.org">numpy.org</a>)._

_Use code cell comments to describe the packages being imported._

In [ ]:
# general python packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

# LSST package for TAP queries
from lsst.rsp import get_tap_service, retrieve_query

# LSST package for Butler queries
import lsst.daf.butler as dafButler

# LSST package for image display
import lsst.afw.display as afwDisplay

from matplotlib import colormaps



### 1.2 Define Functions and Parameters

Import some functions from Pedro

In [ ]:
import numpy as np 
from scipy.interpolate import CubicSpline
from scipy.optimize import leastsq
#Constants

A = [3.332, 1.862]
B = [0.631, 1.218]
C = [0.986, 0.238]

#values taken from sbpy for convenience

alpha_12 = np.deg2rad([7.5, 30., 60, 90, 120, 150])

phi_1_sp = [7.5e-1, 3.3486016e-1, 1.3410560e-1, 5.1104756e-2, 2.1465687e-2, 3.6396989e-3]
phi_1_derivs = [-1.9098593, -9.1328612e-2]

phi_2_sp = [9.25e-1, 6.2884169e-1, 3.1755495e-1, 1.2716367e-1, 2.2373903e-2, 1.6505689e-4]
phi_2_derivs = [-5.7295780e-1, -8.6573138e-8]

alpha_3 = np.deg2rad([0.0, 0.3, 1., 2., 4., 8., 12., 20., 30.])

phi_3_sp = [1., 8.3381185e-1, 5.7735424e-1, 4.2144772e-1, 2.3174230e-1, 1.0348178e-1, 6.1733473e-2, 1.6107006e-2, 0.]
phi_3_derivs = [-1.0630097, 0]


phi_1 = CubicSpline(alpha_12, phi_1_sp, bc_type=((1,phi_1_derivs[0]),(1,phi_1_derivs[1])))
phi_2 = CubicSpline(alpha_12, phi_2_sp, bc_type=((1,phi_2_derivs[0]),(1,phi_2_derivs[1])))
phi_3 = CubicSpline(alpha_3, phi_3_sp, bc_type=((1,phi_3_derivs[0]),(1,phi_3_derivs[1])))


def HG_model(phase, params):
    """
    Reference: Bowell et al. (1989)
    This is the oldest 2 parameter model. 
    """
    sin_a = np.sin(phase)
    tan_ah = np.tan(phase/2)
    
    W = np.exp(-90.56 * tan_ah * tan_ah)    
    scale_sina = sin_a/(0.119 + 1.341*sin_a - 0.754*sin_a*sin_a)
    
    phi_1_S = 1 - C[0] * scale_sina
    phi_2_S = 1 - C[1] * scale_sina
    
    phi_1_L = np.exp(-A[0] * np.power(tan_ah, B[0]))
    phi_2_L = np.exp(-A[1] * np.power(tan_ah, B[1]))
    
    phi_1 = W * phi_1_S + (1-W) * phi_1_L
    phi_2 = W * phi_2_S + (1-W) * phi_2_L
    return params[0] - 2.5*np.log10((1-params[1])* phi_1 + (params[1]) * phi_2) 



def HG1G2_model(phase, params):
    """
    Reference: Muinonen et al. (2010) https://ui.adsabs.harvard.edu/abs/2010Icar..209..542M/abstract
    This is the 3 parameter model, which works best when you have a sufficiently long phaseangle coverage.
    """

    tan_ah = np.tan(phase/2)

    phi_1_ev = phi_1(phase)  
    phi_2_ev = phi_2(phase)  
    phi_3_ev = phi_3(phase)  

    msk = phase < 7.5 * np.pi/180

    phi_1_ev[msk] = 1-6*phase[msk]/np.pi 
    phi_2_ev[msk] = 1- 9 * phase[msk]/(5*np.pi)

    phi_3_ev[phase > np.pi/6] = 0

    return params[0] - 2.5 * np.log10(params[1] * phi_1_ev + params[2] * phi_2_ev + (1-params[1]-params[2]) * phi_3_ev)


def HG12_model(phase, params): 
    """
    Reference:
    Muinonen et al. (2010) https://ui.adsabs.harvard.edu/abs/2010Icar..209..542M/abstract
    This is a simplified version of HG1G2. It's more useful when phaseangle coverage is shorter.
    """
    if params[1] >= 0.2:
        G1 = +0.9529*params[1] + 0.02162 
        G2 = -0.6125*params[1] + 0.5572
    else:
        G1 = +0.7527*params[1] + 0.06164  
        G2 = -0.9612*params[1] + 0.6270

    return HG1G2_model(phase, [params[0], G1, G2])


def chi2(params, mag, phase, mag_err, model):
    pred = model(phase, params)
    return (mag - pred)/mag_err


def fit(mag, phase, sigma, model=HG12_model, params=[0.1]):
    phase = np.deg2rad(phase)

    sol = leastsq(chi2, [mag[0]] + params,  (mag, phase, sigma, model), full_output = True)

    return sol

In [ ]:
def fit_models(mag, magSigma, phaseAngle, tdist, rdist, verbose=False):

    # correct the mag to 1AU distance
    dmag = -5. * np.log10(tdist*rdist)
    mag = mag + dmag

    #double check if this is needed
    #phaseAngle = np.deg2rad(phaseAngle)

    # now we'll fit using each one of the HG, HG12 and HG1G2 models and store these in a dictionary of dictionaries
    solutions = {}

    #Let's do HG first
    sol_HG = fit(mag, phaseAngle, magSigma, model=HG_model)

    solutions['HG'] = {}
    # save np.nan values when the fit has not been converged
    try:
        solutions['HG']['chi2'] = np.sum(sol_HG[2]['fvec']**2)
        solutions['HG']['H'] = sol_HG[0][0]
        solutions['HG']['G'] = sol_HG[0][1]
        solutions['HG']['H_err'] = np.sqrt(sol_HG[1][0,0])
        solutions['HG']['G_err'] = np.sqrt(sol_HG[1][1,1])
        solutions['HG']['cov'] = sol_HG[1]
    except TypeError:
        if verbose:
            print('HG model is not converging')
        solutions['HG']['chi2'] = np.nan
        solutions['HG']['H'] = np.nan
        solutions['HG']['G'] = np.nan
        solutions['HG']['H_err'] = np.nan
        solutions['HG']['G_err'] = np.nan
        solutions['HG']['cov'] = np.nan
            

    # now HG12
    sol_HG12 = fit(mag, phaseAngle, magSigma, model=HG12_model)

    solutions['HG12'] = {}
    # save np.nan values when the fit has not been converged
    try:
        solutions['HG12']['chi2'] = np.sum(sol_HG12[2]['fvec']**2)
        solutions['HG12']['H'] = sol_HG12[0][0]
        solutions['HG12']['G12'] = sol_HG12[0][1]
        solutions['HG12']['H_err'] = np.sqrt(sol_HG12[1][0,0])
        solutions['HG12']['G12_err'] = np.sqrt(sol_HG12[1][1,1])
        solutions['HG12']['cov'] = sol_HG12[1]
    except TypeError:
        if verbose:
            print('HG12 model is not converging')
        solutions['HG12']['chi2'] = np.nan
        solutions['HG12']['H'] = np.nan
        solutions['HG12']['G12'] = np.nan
        solutions['HG12']['H_err'] = np.nan
        solutions['HG12']['G12_err'] = np.nan
        solutions['HG12']['cov'] = np.nan

    # finally, HG1G2 - note this returns an extra parameter

    # now HG12, let's tell the code we need that extra parameter
    sol_HG1G2 = fit(mag, phaseAngle, magSigma, model=HG1G2_model, params=[0.1, 0.1])
    
    solutions['HG1G2'] = {}
    # save np.nan values when the fit has not been converged
    try:
        solutions['HG1G2']['chi2'] = np.sum(sol_HG1G2[2]['fvec']**2)
        solutions['HG1G2']['H'] = sol_HG1G2[0][0]
        solutions['HG1G2']['G1'] = sol_HG1G2[0][1]
        solutions['HG1G2']['G2'] = sol_HG1G2[0][1]
        solutions['HG1G2']['H_err'] = np.sqrt(sol_HG1G2[1][0,0])
        solutions['HG1G2']['G1_err'] = np.sqrt(sol_HG1G2[1][1,1])
        solutions['HG1G2']['G2_err'] = np.sqrt(sol_HG1G2[1][2,2])
        solutions['HG1G2']['cov'] = sol_HG1G2[1]
    except TypeError:
        if verbose: 
            print('HG1G2 model is not converging')
        solutions['HG1G2']['chi2'] = np.nan
        solutions['HG1G2']['H'] = np.nan
        solutions['HG1G2']['G1'] = np.nan
        solutions['HG1G2']['G2'] = np.nan
        solutions['HG1G2']['H_err'] = np.nan
        solutions['HG1G2']['G1_err'] = np.nan
        solutions['HG1G2']['G2_err'] = np.nan
        solutions['HG1G2']['cov'] = np.nan
    
    
    return solutions

## 2. Querying the DP0.3 tables and fitting phase curves of SS objects


### 2.1 Create the Rubin TAP Service Client

Get an instance of the TAP service, and assert that it exists.Get an instance of the TAP service, and assert that it exists.

In [ ]:
service = get_tap_service("ssotap")
assert service is not None

### 2.2 Querying the DP0.3 SSObject and MPCORB catalogs

NOTE!!

To ensure to have sufficient data points over a long enough phase angle baseline
as well as to keep a reasonable run time for this tutorial,
we select objects with number of objects > 3000 and arc > 3000 days. I came up with these numbers fter performing some run time tests. Especially, the fitting part takes long. 

For phase curve fitting, we need apparent magnitudes & uncertainties, phase angle topocentric (tdist) and heliocentric (rdist) distances & uncertainties (but they are 0; from SSSource). fitHG12 returns the absmag and phase coeff (G12) These will all be per band. 

In the ssObjectTable, the column G12 is correct in that the final table
should contain the parameter G12, however, the existing version of the column
actually contains parameter G (and therefore should be compared to the HG_model
not HG12 model. The plan is to replace the contents of G12 column with actual G12
fit parameters in the final version


What is the absolute magnitude? The measured magnitude (apparent magnitude) of a Solar System object depends on the incident light from the Sun, in addition to the distance to the observer (as is usual for sources outside the Solar System). So to convert from the apparent magnitude to the reduced magnitude, one factors in this reduction in flux, conventionally bringing both distances to 1 au so the convertion becomes a factor of -5log10 ( heliocentric distance * topocentric distance). There is also the question of the illumination angle, so that a higher phase angle means that a smaller fraction of the object's surface is being illuminated (I think that link @Yumi Choi sent earlier this week is the most accessible I've seen). The absolute magnitude is the magnitude at 1 au from the Sun, 1 au from the observer and at 0 phase angle (note this is an unphysical situation: an object will never satisfy these 3 conditions simultaneously). Typically then one fits the absolute magnitude with an assumed phase curve model (eg the HG or HG12 models). That Muinonen et al (2010) paper goes over in detail.

In [ ]:
nobs_thrh = '3000'
arc_thrh = '3000'

In [ ]:
# Getting a table of columns for unique objects with number of observations > nobs_thrh and arc longer than arc_thrh
query = """
SELECT 
    mpc.ssObjectId, mpc.q, sso.arc, sso.numObs, 
    sso.uH, sso.uHerr, sso.uG12, sso.uG12err, 
    sso.gH, sso.gHerr, sso.gG12, sso.gG12err, 
    sso.rH, sso.rHerr, sso.rG12, sso.rG12err, 
    sso.iH, sso.iHerr, sso.iG12, sso.iG12err, 
    sso.zH, sso.zHerr, sso.zG12, sso.zG12err, 
    sso.yH, sso.yHerr, sso.yG12, sso.yG12err 
FROM 
    dp03_catalogs.MPCORB as mpc 
INNER JOIN dp03_catalogs.SSObject as sso 
ON mpc.ssObjectId = sso.ssObjectId 
WHERE sso.numObs > {} AND sso.arc > {} ORDER by sso.ssObjectId 
""".format(nobs_thrh, arc_thrh)

df_uniqueObj = service.search(query).to_table()
df_uniqueObj

### 2.3 Querying the DP0.3 DiaSource and SSSource catalogs

Getting a table for time series data for unique objects with number of observations > nobs_thrh and arc longer than arc_thrh
This query usually takes ~30 sec, but could run a bit longer when traffic is busy.

In [ ]:
query = """
SELECT 
    dia.ssObjectId, dia.diaSourceId, dia.mag,
    dia.magSigma, dia.filter, dia.midPointTai, 
    sss.phaseAngle, sss.topocentricDist, sss.heliocentricDist
FROM 
    dp03_catalogs.DiaSource as dia
INNER JOIN 
    dp03_catalogs.SSSource as sss
ON 
    dia.diaSourceId = sss.diaSourceId
WHERE 
    dia.ssObjectId 
    IN (
        SELECT sso.ssObjectId
        FROM dp03_catalogs.SSObject as sso 
        WHERE sso.numObs > {} AND sso.arc > {}
        )
ORDER by dia.ssObjectId 
""".format(nobs_thrh, arc_thrh)

df_indivObs = service.search(query).to_table()
df_indivObs

Double check if the number of unique objects in df_indivObs equals to that of df_uniqueObj.

In [ ]:
assert len(df_uniqueObj) == len(np.unique(df_indivObs['ssObjectId']))

### 2.4 Fitting phase curve per filter per unique object

Compute reduced magnitude for each observation and add it as a column to the df_indivObs

In [ ]:
dmag = -5. * np.log10(df_indivObs['topocentricDist']*df_indivObs['heliocentricDist'])
reduced_mag = df_indivObs['mag'] + dmag

df_indivObs.add_column(reduced_mag, name='reducedMag')

Fitting phase curve per filter per unique object using three different fitting functions. This part takes ~6 min for 269 unique objects unless we parallelize the code. Note that I selected the medium server size.

In [ ]:
fitted_array = []

for iobj in df_uniqueObj['ssObjectId']:
    idx = df_indivObs['ssObjectId'] == iobj
    df_tmp = df_indivObs[idx]
    filts = np.unique(df_tmp['filter'])
    for ifilt in filts:
        idx_filt = df_tmp['filter'] == ifilt
        nobs_ifilt = len(df_tmp[idx_filt])
    
        # number of observations needs to be greater than the number of fit parameter, which is 3.
        if nobs_ifilt > 3:
            x_fitted = fit_models(df_tmp['mag'][idx_filt], 
                                  df_tmp['magSigma'][idx_filt], 
                                  df_tmp['phaseAngle'][idx_filt],
                                  df_tmp['topocentricDist'][idx_filt],  
                                  df_tmp['heliocentricDist'][idx_filt],
                    )
            fitted_array.append([iobj, ifilt, x_fitted])

In [ ]:
results = pd.DataFrame(fitted_array)
results.columns=['ssObjectId', 'fname', 'fit_param']

Converting the fit parameter dictionary to individual columns in a pandas dataframe makes it easy to read each parameters. But it also takes some time. It might be great if we can come up with a more efficient way of storing and accessing all the necessary columns for making plots and doing analysis in later parts of this tutorial.

In [ ]:
results = results[['ssObjectId', 'fname']].join(pd.json_normalize(results.fit_param))

In [ ]:
plt.figure(figsize=(18,5))

plt.subplot(131)
h = plt.hist2d(results[results.fname=='g']['HG.H'], results[results.fname=='g']['HG.G'], bins=100)
plt.title('g-band')
plt.xlabel('H')
plt.ylabel('G')

plt.subplot(132)
h = plt.hist2d(results[results.fname=='r']['HG.H'], results[results.fname=='r']['HG.G'], bins=100)
plt.title('r-band')
plt.xlabel('H')
plt.ylabel('G')

plt.subplot(133)
h = plt.hist2d(results[results.fname=='i']['HG.H'], results[results.fname=='i']['HG.G'], bins=100)
plt.title('i-band')
plt.xlabel('H')
plt.ylabel('G')

In [ ]:
# Plot example phase curve for a single object
sId = df_uniqueObj['ssObjectId'][0]
df_tmp = df_indivObs[df_indivObs['ssObjectId'] == sId]

for ifilt in filts:
    idx = df_tmp['filter'] == ifilt
    
    plt.errorbar(df_tmp['phaseAngle'][idx], df_tmp['reducedMag'][idx], yerr=df_tmp['magSigma'][idx], fmt='o', label=ifilt)

    phases = np.linspace(0,90,100)
    HG_mag = HG_model(np.deg2rad(phases),
                     [results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG.H'].values,
                      results[(results.ssObjectId == sId) & (results.fname == ifilt)]['HG.G'].values])
    # HG12_mag = HG12_model(np.deg2rad(phases), [x['HG12']['H'], x['HG12']['G12']])
    # HG_mag = HG_model(np.deg2rad(phases), [x['HG']['H'], x['HG']['G']])
    # HG1G2_mag = HG1G2_model(np.deg2rad(phases), [x['HG1G2']['H'], x['HG1G2']['G1'], x['HG1G2']['G2']])

    plt.plot(phases, HG_mag, label ='HG 2-parameter model')
    
plt.xlim(df_tmp['phaseAngle'].min()-5, df_tmp['phaseAngle'].max()+5)
plt.ylim(df_tmp['reducedMag'].max()+0.5, df_tmp['reducedMag'].min()-0.5)
plt.xlabel('Phase Angle [deg]')
plt.ylabel('Reduced magnitude [mag]')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.title('ssObjectId = %d' % sId)

## Junk Code Below

In [ ]:
whr = np.where(df['filter'] == 'r')[0]
whz = np.where(df['filter'] == 'z')[0]
whi = np.where(df['filter'] == 'i')[0]
whg = np.where(df['filter'] == 'g')[0]
print(df['magSigma'][whr])

whband = whi

# Include these for distance correction to Reduced magnitude:
tdist = df['topocentricDist']
rdist = df['heliocentricDist']
dmag = -5. * np.log10(tdist*rdist)


#plt.errorbar(df['phaseAngle'][whr], df['mag'][whr]+dmag[whr], yerr=df['magSigma'][whr],fmt='o', alpha=.8,label='r band')
#plt.errorbar(df['phaseAngle'][whz], df['mag'][whz]+dmag[whz], yerr=df['magSigma'][whz],fmt='o', alpha=.8,label='z')
plt.errorbar(df['phaseAngle'][whband], df['mag'][whband]+dmag[whband], yerr=df['magSigma'][whband],fmt='ro', alpha=.8,label='i')
#plt.errorbar(df['phaseAngle'][whg], df['mag'][whg]+dmag[whg], yerr=df['magSigma'][whg],fmt='ok', alpha=.8,label='g')

#plt.ylim((np.max(df['mag'][whr]+dmag[whr])+.1,np.min(df['mag'][whr]+dmag[whr])-1.5))
plt.ylim(np.max(df['mag'][whband]+dmag[whband])+0.5, np.min(df['mag'][whband]+dmag[whband])-0.5)
plt.xlim(np.min(df['phaseAngle'][whband])-5, np.max(df['phaseAngle'][whband])+5)
plt.xlabel('Phase Angle [deg]')
plt.ylabel('Reduced magnitude')

#print(df['mag'][whr], df['phaseAngle'][whr])
#print(len(whr))
#print(np.unique(df['filter']))


#x = fit_models(df['mag'][whr], df['magSigma'][whr], df['phaseAngle'][whr], df['topocentricDist'][whr], df['heliocentricDist'][whr])
x = fit_models(df['mag'][whband], df['magSigma'][whband], df['phaseAngle'][whband], df['topocentricDist'][whband], df['heliocentricDist'][whband])


phases = np.linspace(0,90,100)
HG12_mag = HG12_model(np.deg2rad(phases), [x['HG12']['H'], x['HG12']['G12']])
HG_mag = HG_model(np.deg2rad(phases), [x['HG']['H'], x['HG']['G']])
HG1G2_mag = HG1G2_model(np.deg2rad(phases), [x['HG1G2']['H'], x['HG1G2']['G1'], x['HG1G2']['G2']])

plt.plot(phases, HG_mag, label ='HG 2 parameter model')
#plt.plot(phases, HG12_mag, label='HG12 2 parameters model')
#plt.plot(phases, HG1G2_mag, label='HG1G2 3 parameter model')

# first character is the filter.
# NOTE: In the ssObjectTable, the column G12 is correct in that the final table
# should contain the parameter G12, however, the existing version of the column
# actually contains parameter G (and therefore should be compared to the HG_model
# not HG12 model. The plan is to replace the contents of G12 column with actual G12
# fit parameters in the final version
query2 = "SELECT ssObjectId, rH, rHerr, rG12, rG12err, " + \
         "iH, iHerr, iG12, iG12err " + \
        "FROM dp03_catalogs.SSObject  " + \
        "WHERE ssObjectId = " + str(ssObjId) + " "

df2 = service.search(query2).to_table() # .to_pandas()
print(df2)

#HG12_mag_sso = HG12_model(np.deg2rad(phases), [df2['rH'], df2['rG12']])
#HG12_mag_sso = HG12_model(np.deg2rad(phases), [df2['iH'], df2['iG12']])
#HG_mag = HG_model(np.deg2rad(phases), [x['HG']['H'], x['HG']['G']])
HG_mag_sso = HG_model(np.deg2rad(phases), [df2['iH'], df2['iG12']])
#HG1G2_mag = HG1G2_model(np.deg2rad(phases), [x['HG1G2']['H'], x['HG1G2']['G1'], x['HG1G2']['G2']])

plt.plot(phases, HG_mag_sso, label ='Model in SSObject table')

plt.legend()


In [ ]:

#query = "SELECT mpc.ssObjectId, mpc.q, sso.ssObjectId, sso.arc, dia.mag," + \
#        "dia.magSigma, dia.filter, sss.phaseAngle, sss.topocentricDist, " + \
#        "sss.heliocentricDist, dia.midPointTai " + \
#        "FROM dp03_catalogs.MPCORB as mpc " + \
#        "JOIN dp03_catalogs.SSObject as sso " + \
#        "ON mpc.ssObjectId = sso.ssObjectId " + \
#        "JOIN dp03_catalogs.DiaSource as dia " + \
#        "ON mpc.ssObjectId = dia.ssObjectId " + \
#        "JOIN dp03_catalogs.SSSource as sss " + \
#        "ON mpc.ssObjectId = sss.ssObjectId " + \
#        "WHERE sss.ssObjectId = 3728357697728353090"


In [ ]:
#This is joining all 4 tables at once. It's convient b/c it returns a single table with all columns we need from the 4 tables, but takes longer than separting two queries 

# Getting a list of ssObjectID for sources with number of observations > 1000 and arc longer than 3000 days 
# to have sufficient phase angle coverage.
# query0 = """
# SELECT 
#     mpc.ssObjectId, mpc.q, sso.arc, sso.numObs, 
#     sso.uH, sso.uHerr, sso.uG12, sso.uG12err, 
#     sso.gH, sso.gHerr, sso.gG12, sso.gG12err, 
#     sso.rH, sso.rHerr, sso.rG12, sso.rG12err, 
#     sso.iH, sso.iHerr, sso.iG12, sso.iG12err, 
#     sso.zH, sso.zHerr, sso.zG12, sso.zG12err, 
#     sso.yH, sso.yHerr, sso.yG12, sso.yG12err, 
#     sss.diaSourceId, dia.mag
# FROM 
#     dp03_catalogs.MPCORB as mpc 
# INNER JOIN dp03_catalogs.SSObject as sso 
# ON mpc.ssObjectId = sso.ssObjectId 
# INNER JOIN dp03_catalogs.DiaSource as dia
# ON sso.ssObjectId = dia.ssObjectId
# INNER JOIN dp03_catalogs.SSSource as sss
# ON dia.diaSourceId = sss.diaSourceId
# WHERE sso.numObs > 2000 AND sso.arc > 3000 
    
# """
# df0 = service.search(query0).to_table()
# df0

In [ ]:
# Object from Meg's notebook: -9223369546614897710
#ssObjId = df0['ssObjectId'] 

# diaObjectId is the ID that links the individual measurements between DiaObject and SSSource,
# so you have to match on this id for unique measurement matching. 
# To then match this object to MPCORB and SSObject, you have
# to link on the SSObjectID (hwich is unique per solar system object)

# query = "SELECT dia.diaSourceId, sss.diaSourceId, dia.mag, " + \
#         "dia.magSigma, dia.filter, sss.phaseAngle, sss.topocentricDist, " + \
#         "sss.heliocentricDist, dia.midPointTai " + \
#         "FROM dp03_catalogs.DiaSource as dia " + \
#         "JOIN dp03_catalogs.SSSource as sss " + \
#         "ON dia.diaSourceId = sss.diaSourceId " + \
#         "WHERE dia.ssObjectId IN {} ORDER by sss.diaSourceId".format(tuple(df0['ssObjectId']))


# df = service.search(query).to_table()
# df

In [ ]:
fit_param = {}

for iobj in df_uniqueObj['ssObjectId']:
    idx = df_indivObs['ssObjectId'] == iobj
    df_tmp = df_indivObs[idx]
    filts = np.unique(df_tmp['filter'])
    x_filt = {}
    for ifilt in filts:
        idx_filt = df_tmp['filter'] == ifilt
        nobs_ifilt = len(df_tmp[idx_filt])
    
        # number of observations needs to be greater than the number of fit parameter, which is 3.
        if nobs_ifilt > 3:
            x_filt[ifilt] = fit_models(df_tmp['mag'][idx_filt], 
                                       df_tmp['magSigma'][idx_filt], 
                                       df_tmp['phaseAngle'][idx_filt],
                                       df_tmp['topocentricDist'][idx_filt],  
                                       df_tmp['heliocentricDist'][idx_filt],
                    )
            
    fit_param[iobj] = x_filt
    